# LLMS-Generator Experiments

For experimentation with the LLMS-Generator Agent.

## Setup

To run this notebook, ensure you've authenticated to Google Cloud and installed the project's dependencies.

To setup the Google Cloud environment:

```bash
source scripts/setup-env.sh
```

Then to install the package dependencies into the virtual environment, use the `uv` tool:

1. From your agent's root directory, run `make install` to set up the virtual environment (`.venv`).
2. In this Jupyter notebook, select the kernel from the `.venv` folder to ensure all dependencies are available.

### Import Libraries

In [1]:
import os

import vertexai
from dotenv import load_dotenv
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.auth import default
from google.genai.types import Content, Part
from IPython.display import Markdown, display

### Load Environment Variables

In [2]:
dotenv_path = os.path.abspath('../.env')

if os.path.exists(dotenv_path):
    print(f"Loading environment variables from: {dotenv_path}")
    load_dotenv(dotenv_path=dotenv_path)
else:
    print(f"Warning: .env file not found at {dotenv_path}")

staging_project_id = os.getenv("GOOGLE_CLOUD_STAGING_PROJECT")
if staging_project_id:
    os.environ["GOOGLE_CLOUD_PROJECT"] = staging_project_id
    print(f"Set GOOGLE_CLOUD_PROJECT environment variable to: {staging_project_id}")


Loading environment variables from: /home/darren/localdev/Python/llms-gen/.env
Set GOOGLE_CLOUD_PROJECT environment variable to: scratch-dev-428715


### Ensure We're Authenticating to the Right Project

In [3]:
credentials, project_id = default()  # To use ADC
vertexai.init(project=staging_project_id, location="europe-west4", credentials=credentials) # type: ignore

## Running the Agent

In [4]:
from llms_gen_agent.agent import root_agent

18:52:55.750:llms_gen_agent - INFO: Logger initialised for llms_gen_agent.
18:52:55.750:llms_gen_agent - DEBUG: DEBUG level logging enabled.
18:52:56.337:llms_gen_agent - DEBUG: agent_name set to llms_gen_agent
18:52:56.338:llms_gen_agent - DEBUG: project_id set to scratch-dev-428715
18:52:56.338:llms_gen_agent - DEBUG: location set to global
18:52:56.338:llms_gen_agent - DEBUG: model set to gemini-2.5-flash
18:52:56.339:llms_gen_agent - DEBUG: genai_use_vertexai set to True


In [5]:
# Session and Runner
APP_NAME = "generate_llms_client"
USER_ID="test_user"
SESSION_ID="test_session"
TEST_REPO_PATH = "/home/darren/localdev/gcp/adk-docs" # You can change this to any repository you want to test

async def setup_session_and_runner():
    session_service = InMemorySessionService()
    session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
    runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)
    return session, runner

In [ ]:
# Agent Interaction
async def call_agent_async(query):
    content = Content(role='user', parts=[Part(text=query)])
    _, runner = await setup_session_and_runner()
    events = runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

    async for event in events:
        if function_calls := event.get_function_calls():
            tool_name = function_calls[0].name
            display(Markdown(f"_Using tool {tool_name}..._"))
        elif event.actions and event.actions.transfer_to_agent:
            personality_name = event.actions.transfer_to_agent
            display(Markdown(f"_Delegating to agent: {personality_name}..._"))
        elif event.is_final_response() and event.content and event.content.parts:
            final_response_content = event.content.parts[0].text
            display(Markdown("### Final Message"))
            display(Markdown(final_response_content))

        # For debugging, print the raw type and content to the console
        print(f"DEBUG: Full Event: {event}")

In [7]:
query = f"Generate the llms.txt file for the repository at {TEST_REPO_PATH}"
await call_agent_async(query)

_Using tool discover_files..._

18:52:58.003:llms_gen_agent - DEBUG: Entering discover_files with repo_path: /home/darren/localdev/gcp/adk-docs
18:52:58.005:llms_gen_agent - DEBUG: Dirs
:/home/darren/localdev/gcp/adk-docs
/home/darren/localdev/gcp/adk-docs/docs
/home/darren/localdev/gcp/adk-docs/docs/deploy
/home/darren/localdev/gcp/adk-docs/docs/sessions
/home/darren/localdev/gcp/adk-docs/docs/tutorials
/home/darren/localdev/gcp/adk-docs/docs/streaming
/home/darren/localdev/gcp/adk-docs/docs/streaming/dev-guide
/home/darren/localdev/gcp/adk-docs/docs/callbacks
/home/darren/localdev/gcp/adk-docs/docs/context
/home/darren/localdev/gcp/adk-docs/docs/a2a
/home/darren/localdev/gcp/adk-docs/docs/runtime
/home/darren/localdev/gcp/adk-docs/docs/safety
/home/darren/localdev/gcp/adk-docs/docs/observability
/home/darren/localdev/gcp/adk-docs/docs/events
/home/darren/localdev/gcp/adk-docs/docs/tools
/home/darren/localdev/gcp/adk-docs/docs/mcp
/home/darren/localdev/gcp/adk-docs/docs/evaluate
/home/darren/localdev/gcp/adk-docs/do

DEBUG: Full Event: content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'repo_path': '/home/darren/localdev/gcp/adk-docs'
        },
        id='adk-1fb86c7d-e6dc-4c42-a478-f561bead29d7',
        name='discover_files'
      ),
      thought_signature=b"\n\xab\x05\x01\x1f\xcc\x85\xb6F\xf2\x9e)\xb9l\xf0\xd3\xcc\x7f\x06\xd3\xd3\x15\xfd\xe0\xdb\xf1#\x85q\x00Q\x84xHq\x8a\te=\x9do \x93r\xba\xcc\xc5gd\x930\xd85\x1a\xb3\xc3\xe1g|\xea6'D\xab\xa9\x84\x86$\x19\xc8\xbb\xc5:\xab~\x90\x89\x18\x9f\xa8\x85\x86x\x97#\x0eX4{#\xe2\r\t>#\x96\xfb...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=21,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=21
    ),
  ],
  promp

_Using tool document_summariser_agent..._

DEBUG: Full Event: content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'request': 'Summarize the discovered files.'
        },
        id='adk-5f764484-5e33-447b-a063-d9ac3c30de15',
        name='document_summariser_agent'
      ),
      thought_signature=b'\n\x9b\x02\x01\x1f\xcc\x85\xb6\xbfb\xd8M#\x11\xa3\xc9\xd8M\x19\x91*\xb3A\x02X\xf0\xcdMp\xde\x94K\xba\x87\x84\xc3iR\x1d\x8a\x0c\x0e\xe7\xaf\xa7\x80\xe2\x04/f\xf7r\xa5\xd7\xb4T;\xbf\xed!Dq\x86\x97\xe7\x0b\xf6\xad_\x03\xe1CA\xa5:\xa3\x12+s#\xd0\xc2\xf5\x1e\x0c\x17D&\x12\xd5\xb4\x92\xef\x0b \\\xe5...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=13,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=

...

18:53:25.792:llms_gen_agent - DEBUG: Entering generate_llms_txt for repo_path: /home/darren/localdev/gcp/adk-docs
18:53:25.793:llms_gen_agent - DEBUG: We have 32 directories.
18:53:25.793:llms_gen_agent - DEBUG: We have 85 files
18:53:25.794:llms_gen_agent - DEBUG: We have 10 sumamries
18:53:25.795:llms_gen_agent - DEBUG: Exiting generate_llms_txt. llms.txt generated at /home/darren/localdev/Python/llms-gen/notebooks/temp/llms.txt


DEBUG: Full Event: content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'doc_summaries': <dict len=10>,
          'repo_path': '/home/darren/localdev/gcp/adk-docs'
        },
        id='adk-42f75c5f-a345-44b7-851a-c66e6a129e96',
        name='generate_llms_txt'
      ),
      thought_signature=b'\n\xba\x08\x01\x1f\xcc\x85\xb6\xdc\xe8L<\xccm[\x8a\x15\xc3-\xd3\x01\xf6\xce\xc5\xb7n\x9f\xbf\x99)\x85\x10n\x85K\xea\x9b-(\\\xe5\x9e)<Of\xcfKGI\xcc\xb4\xca\xb2M\x85\xfa\xc8\xfcbe\xad1\x8c$&\xbaY\xe6\x99V\xee^\xb1]\x91\x8ar\x00\x9f\x05\x84\xf0\x18\x04\xa6pA\xd9F`U$\xda\xaf\x04\xc7...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=1243,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: '

### Final Message

The `llms.txt` file has been successfully created at `/home/darren/localdev/Python/llms-gen/notebooks/temp/llms.txt`.

```
# ADK-Docs Repository Overview

This repository contains the documentation for the Agent Development Kit (ADK), an open-source toolkit designed for building, evaluating, and deploying AI agents. The documentation covers various aspects of ADK, including its core libraries (Python and Java), deployment options, agent configurations, and community resources.

## Key Features and Concepts:

*   **Flexible and Modular Framework**: ADK is designed to be model-agnostic and deployment-agnostic, with a focus on optimization for Gemini and the Google ecosystem.
*   **Multi-Agent Architecture**: Supports the creation of complex multi-agent systems.
*   **Rich Tool Ecosystem**: Provides a wide range of tools for agents to interact with external systems.
*   **Deployment Readiness**: Offers various deployment options for scaling agents in production.
*   **Built-in Evaluation**: Includes features for evaluating agent performance.
*   **Safety and Security**: Emphasizes safety and security in agent development.

## Core Documentation Areas:

### Getting Started:
*   **Installation**: Instructions for setting up ADK in both Python and Java environments.
*   **Quickstarts**: Guides to quickly get started with ADK, including streaming examples.
*   **About**: Provides an overview of the ADK project.
*   **Testing**: Information on testing ADK agents.

### Agents:
*   **LLM Agents**: Details on configuring and using agents powered by Large Language Models.
*   **Custom Agents**: Guidance on creating custom agent implementations.
*   **Multi-Agents**: Information on orchestrating multiple agents.
*   **Agent Configuration**: How to configure various aspects of agents.
*   **Models**: Discusses the integration and use of different models with ADK.
*   **Workflow Agents**:
    *   **Sequential Agents**: Agents that execute tasks in a defined order.
    *   **Parallel Agents**: Agents that perform tasks concurrently.
    *   **Loop Agents**: Agents designed for iterative processes.

### Deployment:
*   **Agent Engine**: Deploying ADK agents to Vertex AI Agent Engine for a fully managed service.
*   **Cloud Run**: Deploying containerized ADK agents to Google Cloud Run.
*   **Google Kubernetes Engine (GKE)**: Deploying ADK agents to GKE for more control and open model support.
*   **Other Container Infrastructures**: Instructions for deploying to Docker or Podman.

### Sessions:
*   **State Management**: The `state` attribute within an ADK `Session` acts as a scratchpad for agents to store and update dynamic information. It supports serializable key-value pairs for personalization, tracking progress, and decision-making. Prefixes (`user:`, `app:`, `temp:`) define the scope and persistence of state variables.
*   **Memory**: How agents manage and utilize memory within sessions.
*   **Express Mode**: Details on using express mode for sessions.

### Streaming:
*   **Custom Streaming**: Implementing custom streaming solutions, including WebSocket-based streaming.
*   **Streaming Tools**: Utilizing tools specifically designed for streaming.
*   **Configuration**: Configuring streaming behavior.
*   **Developer Guide**: A multi-part guide for developing streaming functionalities.

### Tools:
*   **Built-in Tools**: Overview of tools provided out-of-the-box.
*   **Google Cloud Tools**: Tools for interacting with Google Cloud services.
*   **OpenAPI Tools**: Integrating with OpenAPI specifications.
*   **Function Tools**: Creating and using function-based tools.
*   **Third-Party Tools**: Integrating with external tools.
*   **Authentication**: Handling authentication for tools.
*   **Confirmation**: Implementing confirmation mechanisms for tool usage.
*   **Performance**: Optimizing tool performance.
*   **MCP Tools**: Tools related to Multi-Cloud Platform.

### Callbacks:
*   **Types of Callbacks**: Different types of callbacks available in ADK.
*   **Design Patterns and Best Practices**: Guidelines for effective callback implementation.

### Context:
*   Information related to managing and utilizing context within agents.

### A2A (Agent-to-Agent Communication):
*   **Introduction**: Overview of agent-to-agent communication.
*   **Quickstart Exposing**: How to expose agent functionalities.
*   **Quickstart Consuming**: How to consume functionalities from other agents.

### Runtime:
*   **RunConfig**: Configuration options for agent runtime.

### Safety:
*   Information and guidelines related to ensuring safety in agent development.

### Observability:
*   **AgentOps**: Monitoring agent operations.
*   **Cloud Trace**: Utilizing Google Cloud Trace for observability.
*   **Phoenix**: Integration with Phoenix for observability.
*   **Weave**: Using Weave for observability.
*   **Arize AX**: Integration with Arize AX for observability.
*   **Logging**: Best practices and configurations for logging.

### Events:
*   Details on handling and processing events within ADK.

### MCP (Multi-Cloud Platform):
*   Information specific to Multi-Cloud Platform integrations.

### Evaluate:
*   Tools and methodologies for evaluating agent performance.

### API Reference:
*   **Java API**: Reference for the Java API, including legal notices for third-party libraries like jQuery and DejaVu fonts.
*   **REST API**: Reference for the REST API.

### Artifacts:
*   Management and use of artifacts within ADK.

### Plugins:
*   Information on developing and using plugins.

### Grounding:
*   **Vertex AI Search Grounding**: Grounding agents using Vertex AI Search.
*   **Google Search Grounding**: Grounding agents using Google Search.

## Contributing:

*   **Contribution Guidelines**: Detailed instructions on how to contribute to the ADK project, including signing the Contributor License Agreement (CLA), reviewing community guidelines, and the workflow for submitting code reviews via Pull Requests.
*   **GitHub Issues**: Reporting issues and suggesting enhancements.
*   **Documentation Improvements**: Contributing to the ADK documentation.

## Community:

*   **Translations**: Community-provided translations of the documentation (Chinese, Korean, Japanese).
*   **Tutorials, Guides, Blog Posts, Videos**: Resources showcasing ADK features, use cases, and integrations.

## Examples:

*   **Java Demos**: Example Java applications, including a patent search agent.
*   **Python Tutorials**: Tutorials for Python-based ADK development.

## Repository Structure:

*   `.github/ISSUE_TEMPLATE`: Templates for bug reports and feature requests.
*   `docs/`: Contains the main documentation files, organized by topic.
*   `examples/`: Provides example code for both Java and Python.
*   `CONTRIBUTING.md`: Guidelines for contributing to the project.
*   `README.md`: General overview and installation instructions.
```


DEBUG: Full Event: content=Content(
  parts=[
    Part(
      text="""The `llms.txt` file has been successfully created at `/home/darren/localdev/Python/llms-gen/notebooks/temp/llms.txt`.

```
# ADK-Docs Repository Overview

This repository contains the documentation for the Agent Development Kit (ADK), an open-source toolkit designed for building, evaluating, and deploying AI agents. The documentation covers various aspects of ADK, including its core libraries (Python and Java), deployment options, agent configurations, and community resources.

## Key Features and Concepts:

*   **Flexible and Modular Framework**: ADK is designed to be model-agnostic and deployment-agnostic, with a focus on optimization for Gemini and the Google ecosystem.
*   **Multi-Agent Architecture**: Supports the creation of complex multi-agent systems.
*   **Rich Tool Ecosystem**: Provides a wide range of tools for agents to interact with external systems.
*   **Deployment Readiness**: Offers various deploymen